In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.preprocessing import MinMaxScaler

In [2]:
model = Sequential()
model.add(Dense(32, input_dim=10, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(loss="mean_squared_error", optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.load_weights('./Model_check_point/final_checkpoint_20210712_2255.ckpt')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001E685A61250> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001E685A29E50>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001E685BFC550> and <tensorflow.python.keras.layers.core.Dense object at 0x000001E685A61250>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001E68D1EBC40> and <tensorflow.python.keras.layers.core.Dense object at 0x000001E685BFC550>).


ValueError: Shapes (16,) and (32,) are incompatible

In [4]:
data = pd.read_csv('./data/data_frame_v2.csv', index_col=None)
print(data.columns)
data

Index(['year', 'month', 'ID', 'data_block_num', 'shop_id', 'item_id',
       'item_category_id', 'item_price', 'category_occ',
       'total_mean_item_price', 'item_cnt_month'],
      dtype='object')


,year,month,ID,data_block_num,shop_id,item_id,item_category_id,item_price,category_occ,total_mean_item_price,item_cnt_month
0,2013,1,5122,0,4,5823,35,2500.00,162,2635.116266,1
1,2013,1,5127,0,4,5643,2,2330.25,43,2539.082418,3
2,2013,1,5142,0,4,5821,35,599.00,162,822.133789,2
3,2013,1,5143,0,4,5822,35,999.00,162,1064.279381,9
4,2013,1,5152,0,4,5459,55,299.00,1105,330.464615,1
...,...,...,...,...,...,...,...,...,...,...,...
600154,2015,10,213962,33,45,16106,64,799.00,158,573.999197,1
600155,2015,10,214068,33,45,6184,30,498.00,1362,670.072416,1
600156,2015,10,214069,33,45,3985,55,98.00,1105,249.491071,1
600157,2015,10,214187,33,45,5401,19,799.00,677,1626.173585,1


In [5]:
sample = pd.read_csv('./data/sample_submission.csv', index_col=None)
sample

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [6]:
sell = sample.copy()
sell.item_cnt_month = 0
sell.columns = ['ID', 'sell']

# 3개월 동안 팔렸나 확인
data_3m = data[(31 <= data.data_block_num) & (data.data_block_num <= 33)][['data_block_num', 'ID', 'item_cnt_month']]
data_3m = data_3m.groupby('ID').sum()[['item_cnt_month']].reset_index()

sell = pd.merge(sell, data_3m,
                how='left', on='ID')
sell.fillna(0, inplace=True)

sell['sell_3m'] = sell['item_cnt_month'].apply(lambda c: 1 if c>=3 else 0)

sell = sell[['ID', 'sell', 'sell_3m']]

# 같은 월에 팔린 item
data_same = data[data.month == 11].copy()
data_same = data_same[data_same.year < 2015]
data_same = data_same.groupby('ID').sum()[['item_cnt_month']].reset_index()

sell = pd.merge(sell, data_same,
                how='left', on='ID')
sell.fillna(0, inplace=True)

sell['sell_same'] = sell['item_cnt_month'].apply(lambda c: 1 if c>=2 else 0)

sell = sell[['ID', 'sell', 'sell_3m', 'sell_same']]

# 합집합 남기기
sell['sell'] = sell['sell_3m'] + sell['sell_same']
sell['sell'] = sell['sell'].apply(lambda c: 1 if c>0 else 0)

sell = sell[['ID', 'sell']]
sell

,ID,sell
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
214195,214195,0
214196,214196,0
214197,214197,1
214198,214198,0


In [7]:
please1 = data[(31 <= data.data_block_num) & (data.data_block_num <= 33)]
please2 = data[data.month == 11]
please2 = please2[please2.year < 2015]
please = pd.concat([please1, please2])
please

,year,month,ID,data_block_num,shop_id,item_id,item_category_id,item_price,category_occ,total_mean_item_price,item_cnt_month
516031,2015,8,0,31,5,5037,19,999.166667,677,1891.068359,3
516032,2015,8,2,31,5,5233,19,599.000000,677,802.390632,1
516033,2015,8,3,31,5,5232,23,599.000000,569,789.903798,1
516034,2015,8,8,31,5,5319,55,299.000000,1105,298.372458,2
516035,2015,8,10,31,5,4806,30,99.000000,1362,144.479240,5
...,...,...,...,...,...,...,...,...,...,...,...
313237,2014,11,214116,22,45,15545,37,629.000000,350,687.342105,1
313238,2014,11,214151,22,45,7925,7,2999.000000,28,2884.905856,1
313239,2014,11,214152,22,45,6117,30,399.000000,1362,341.470427,1
313240,2014,11,214156,22,45,3316,19,199.000000,677,458.607202,1


In [8]:
please = please.groupby('ID').mean()[['item_price']].reset_index()
please

,ID,item_price
0,0,1449.222222
1,2,932.333333
2,3,599.000000
3,5,1124.250000
4,6,3915.666667
...,...,...
70112,214180,149.000000
70113,214181,899.000000
70114,214187,799.000000
70115,214195,99.000000


In [9]:
sell = pd.merge(sell, please, how='left', on='ID')
sell.fillna(0, inplace=True)
sell

,ID,sell,item_price
0,0,1,1449.222222
1,1,0,0.000000
2,2,1,932.333333
3,3,0,599.000000
4,4,0,0.000000
...,...,...,...
214195,214195,0,99.000000
214196,214196,0,0.000000
214197,214197,1,199.000000
214198,214198,0,0.000000


In [10]:
test_csv = pd.read_csv('./data/test.csv', index_col=None)
test_csv

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [11]:
items_csv = pd.read_csv('./data/items.csv', index_col=None)
items_csv

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


In [12]:
occ_csv = pd.read_csv('./data/occ.csv', index_col=None)
occ_csv.columns = ['item_category_id', 'category_occ']
occ_csv.head()

,item_category_id,category_occ
0,2,43.0
1,3,119.0
2,5,13.0
3,6,84.0
4,7,28.0


In [13]:
tmip = pd.read_csv('./data/total_mean_item_price.csv', index_col=None)
tmip.columns = ['item_id', 'total_mean_item_price']
tmip.head()

,item_id,total_mean_item_price
0,30,324.562531
1,31,573.793088
2,32,250.083706
3,33,249.273016
4,38,2351.537037


In [14]:
x_test2 = sell[sell.sell==1].copy()
x_test2 = x_test2[['ID', 'item_price']]
x_test2['year'] = 2015
x_test2['month'] = 11
x_test2['data_block_num'] = 34

x_test2 = pd.merge(x_test2, test_csv, how='left', on='ID')
x_test2 = pd.merge(x_test2, items_csv[['item_id', 'item_category_id']], how='left', on='item_id')

x_test2 = pd.merge(x_test2, occ_csv, how='left', on='item_category_id')
x_test2 = pd.merge(x_test2, tmip, how='left', on='item_id')

x_test2

,ID,item_price,year,month,data_block_num,shop_id,item_id,item_category_id,category_occ,total_mean_item_price
0,0,1449.222222,2015,11,34,5,5037,19,677.0,1891.068359
1,2,932.333333,2015,11,34,5,5233,19,677.0,802.390632
2,6,3915.666667,2015,11,34,5,5041,20,606.0,3837.086434
3,8,299.000000,2015,11,34,5,5319,55,1105.0,298.372458
4,10,149.000000,2015,11,34,5,4806,30,1362.0,144.479240
...,...,...,...,...,...,...,...,...,...,...
26178,214041,599.000000,2015,11,34,45,6488,28,387.0,514.850994
26179,214068,598.500000,2015,11,34,45,6184,30,1362.0,670.072416
26180,214098,549.000000,2015,11,34,45,5621,2,43.0,547.243750
26181,214103,549.000000,2015,11,34,45,15472,63,177.0,635.375066


In [15]:
col2 = data.columns.to_list()[:10]
x_test2 = x_test2[col2]

In [16]:
x_train = data[data.data_block_num <= 33].values[:, :10]

scaler = MinMaxScaler()
scaler.fit(x_train)
x_test2_scale = scaler.transform(x_test2)

In [17]:
# # y_test에 zero-padding
# y_test = sample[['ID']].copy()
# y_test = pd.merge(y_test, data[data.data_block_num == 33][['ID', 'item_cnt_month']], how='left', on='ID')
# y_test.fillna(0, inplace=True)

# y_pred2에 zero-padding
x_test2_scale = scaler.transform(x_test2.values)

y_pred2_raw = model.predict(x_test2_scale)
y_temp2 = x_test2[['ID']].copy()
y_temp2['item_cnt_month'] = y_pred2_raw

y_pred2 = sample[['ID']].copy()
y_pred2 = pd.merge(y_pred2, y_temp2, how='left', on='ID')
y_pred2.fillna(0, inplace=True)
y_pred2.head(20)

,ID,item_cnt_month
0,0,2.677190
1,1,0.000000
2,2,2.372317
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,14.890006
7,7,0.000000
8,8,2.046912
9,9,0.000000


In [18]:
# 0보다 작은 예측은 0으로 처리
y_pred2['item_cnt_month'] = y_pred2['item_cnt_month'].apply(lambda x: x if x>0 else 0)
y_pred2.head(20)

,ID,item_cnt_month
0,0,2.677190
1,1,0.000000
2,2,2.372317
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,14.890006
7,7,0.000000
8,8,2.046912
9,9,0.000000


In [19]:
y_pred2.to_csv('./data/final_result3.csv', index=None)